In [ ]:
from dataclasses import dataclass
from datetime import datetime

import sklearn.metrics as metrics
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np 
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from tqdm.auto import tqdm
from torchfuzzy import FuzzyLayer, DefuzzyLinearLayer
import os

os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [ ]:
batch_size = 512

nz = 100 
ngf = 32 
ngpu = 1

niter = 50

prefix = f"fuzzy_gan_anomaly_detection"
writer = SummaryWriter(f'runs/mnist/{prefix}_{datetime.now().strftime("%Y%m%d-%H%M%S")}')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Load MNIST

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Lambda(lambda x: x.view(-1, 28, 28) - 0.5),
])

In [ ]:
def get_target_and_mask(target_label):
    """
    Args:
    
    Returns:
    
    """
    
    t = torch.zeros(10)
    t[target_label] = 1.0
    
    return t.to(device) 

In [ ]:
# загружаем обучающую выборку
train_data = datasets.MNIST(
    '~/.pytorch/MNIST_data/', 
    download=True, 
    train=True, 
    transform = transform,
    target_transform = transforms.Lambda(lambda x: get_target_and_mask(x))
)
len(train_data)

In [ ]:
# загружаем тестовую выборку
test_data = datasets.MNIST(
    '~/.pytorch/MNIST_data/', 
    download=True, 
    train=False, 
    transform=transform, 
    target_transform = transforms.Lambda(lambda x: get_target_and_mask(x))
)
len(test_data)

In [ ]:
# Создаем итераторы датасетов
train_loader = torch.utils.data.DataLoader(
    train_data, 
    batch_size=batch_size, 
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_data, 
    batch_size=batch_size, 
    shuffle=False,
)


### Load EMNIST

In [ ]:
emnist_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Lambda(lambda x: x.view(-1, 28, 28) - 0.5),
])

emnist_test = pd.read_csv("./data/EMNIST/emnist-letters.zip")
emnist_y = emnist_test["label"]
emnist_x = emnist_test.drop(labels = ["label"], axis = 1) 
del emnist_test 

emnist_x = emnist_x / 255.0
emnist_x = emnist_x.values.reshape(-1, 28, 28)
emnist_x = [torch.tensor(emnist_transform(a), dtype=torch.float32, device=device) for a in emnist_x]

len(emnist_x)

emnist_mapping = pd.read_csv("./data/EMNIST/emnist-letters-mapping.txt", sep=' ', header=None)
emnist_mapping.columns=("EMNIST","UP","LO")
emnist_mapping["Letter"] = emnist_mapping.apply(lambda row: chr(row["UP"])+chr(row["LO"]), axis=1)
emnist_mapping = dict(zip(emnist_mapping["EMNIST"], emnist_mapping["Letter"]))

emnist_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(torch.stack(emnist_x), torch.Tensor(np.array(emnist_y))), 
    batch_size=batch_size, 
    shuffle=False,
)

## DCGAN Model

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu, nc=1, nz=100, ngf=64):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(    ngf,      nc, kernel_size=1, stride=1, padding=2, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output

netG = Generator(ngpu).to(device)
netG.apply(weights_init)
num_params = sum(p.numel() for p in netG.parameters() if p.requires_grad)
print(f'Number of parameters: {num_params:,}')
#netG.load_state_dict(torch.load('weights/netG_epoch_99.pth'))
print(netG)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu, ndf=64):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(1, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 4, 1, 2, 1, 1, bias=False),
            nn.BatchNorm2d(1),
            # nn.Conv2d(1, 1, 2, 2, 0, bias=False),
            # nn.BatchNorm2d(1),
            # nn.LeakyReLU(0.2, inplace=True),
            # nn.Conv2d(1, 1, 2, 2, 0, bias=False),
            # nn.BatchNorm2d(1),
            nn.Flatten(),
            #FuzzyLayer.from_dimensions(9, 10),
            nn.Linear(16, 2),
            #nn.BatchNorm1d(2),
            #DefuzzyLinearLayer.from_dimensions(100, 1, with_norm=False)
            
            #DefuzzyLinearLayer.from_dimensions(20, 1, with_norm=False)
            #nn.Conv2d(ndf * 4, 1, 4, 2, 1, bias=False),
            #nn.Sigmoid()
        )
        self.latent_dim = 2
        self.fuzzlets = 10
        theta = np.linspace(0, 2*np.pi, self.fuzzlets+1)
        a, b = 0.5 * np.cos(theta), 0.5 * np.sin(theta)

        self.core = nn.Sequential(
            FuzzyLayer.from_centers_and_scales([[x[0],x[1]] for x in zip(a,b)][:-1], [[0.1, 0.1] for x in zip(a,b)][:-1]),
            #nn.BatchNorm1d(self.fuzzlets),
            #nn.Linear(self.fuzzlets, 1, bias=False)
            #DefuzzyLinearLayer.from_dimensions(self.fuzzlets, 1, with_norm=False)
        )
        #self.real = nn.Sequential(
        #    FuzzyLayer.from_dimensions(self.latent_dim, self.fuzzlets)
        #)
        #self.core[0].set_requires_grad_rot(False)
        #self.core[0].set_requires_grad_rot(False)
        #self.fake[0].set_requires_grad_rot(False)

    def forward(self, input):
        batch_size = input.shape[0]
        output = self.main(input)
        c = self.core(output)
        centroids_c = self.core[0].centroids.squeeze(-1)
        average_cenroid = centroids_c.mean(0)

        с_winners = torch.randint(0, self.fuzzlets, (batch_size, )).to(device)
        random_centroids = centroids_c[с_winners]
        firing_of_random = c.gather(1, с_winners.reshape((batch_size, 1))).squeeze()

        return c.sum(-1), (output - average_cenroid).abs().sum(-1), (random_centroids - output).abs().sum(-1)
    
    # def forward2(self, input):
    #     batch_size = input.shape[0]
    #     output = self.main(input)
    #     c = self.core(output)
        
    #     centroids_c = self.core[0].centroids.squeeze(-1)

    #     с_winners = torch.randint(0, self.fuzzlets, (batch_size, )).to(device)
    #     centroids_c = centroids_c[с_winners]
        
    #     c = c.gather(1, с_winners.reshape((batch_size, 1))).squeeze()    
    #     return c, (output - centroids_c).abs().sum(-1)
    
    def freeze_encoder(self):
        #self.core[0].set_requires_grad_rot(False)
        self.main.requires_grad_(False)
    
    def arate(self, input):
        return self.forward(input)[0]
    

netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)
num_params = sum(p.numel() for p in netD.parameters() if p.requires_grad)
print(f'Number of parameters: {num_params:,}')
print(netD)

In [ ]:
inp = torch.rand(13, 1, 28, 28).to(device)
dd = Discriminator(1).to(device)
dd(inp)

## Train

In [ ]:
optimizerD = torch.optim.Adam(netD.parameters(), lr=0.001, betas=(0.5, 0.999))
optimizerG = torch.optim.Adam(netG.parameters(), lr=0.001, betas=(0.5, 0.999))

fixed_noise = torch.randn(64, nz, 1, 1, device=device)

In [ ]:
def keep_eigenvals_positive_loss(d, eps = 1e-3):
    ev = d.core[0].get_transformation_matrix_eigenvals().real.min()
    ev = torch.where(ev > eps, eps, ev)
    return -ev

In [ ]:
def get_test_arate_distr(D):
    firing_levels = []
    lab_true = []
    lab_pred = []

    with torch.no_grad():
        for data, _ in tqdm(test_loader, desc='Test MNIST', disable=True):
            data = data.view((-1,1,28,28)).to(device)
            rates = D.arate(data)
            firing_levels.append(rates.cpu().numpy())
            

    firing_levels = np.concatenate(firing_levels, axis=0)
    for p in firing_levels:
        lab_true.append(1)
        lab_pred.append(p)

    with torch.no_grad():
        for data, labels in tqdm(emnist_loader, desc='Test EMNIST', disable=True):
            data = data.view((-1, 1, 28, 28)).to(device) 
            arate = D.arate(data)
            
            for p in arate.cpu().numpy():
                lab_true.append(0)
                lab_pred.append(p)
                
    
    fpr, tpr, threshold = metrics.roc_curve(lab_true, lab_pred)
    roc_auc = metrics.auc(fpr, tpr)

    return firing_levels, roc_auc

def draw_embeddings(epoch):
    #centroids_r = netD.real[0].centroids.squeeze(-1).cpu().detach().numpy()
    #centroids_f = netD.fake[0].centroids.squeeze(-1).cpu().detach().numpy()
    centroids_core = netD.core[0].centroids.squeeze(-1).cpu().detach().numpy()
    
    embedings = []
    labels_expected = []
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc='Encoding', disable=True):
            data = data.view((-1,1,28,28)).to(device)
            embeding = netD.main(data)
            embedings.append(embeding.cpu().numpy())
            labels_expected.append(np.argmax(target.squeeze(1).cpu().numpy(), axis=1))
    embedings = np.concatenate(embedings, axis=0)
    labels_expected = np.concatenate(labels_expected, axis=0)

    embedings_fake = []

    with torch.no_grad():
        batch_size = 256
        latent_size = 100
        
        fixed_noise = torch.randn(batch_size, latent_size, 1, 1)
        if torch.cuda.is_available():
            fixed_noise = fixed_noise.cuda()
        fake_images = netG(fixed_noise)
        embeding = netD.main(fake_images)
        embedings_fake.append(embeding.cpu().numpy())

    embedings_fake = np.concatenate(embedings_fake, axis=0)    

    fig = plt.figure(figsize=(6, 6))
    R, C = 1, 1
    cnt = 1
    for i in range(1):
        plt.subplot(R, C, cnt)
        cnt += 1
        plt.scatter(embedings_fake[:, 2*i], embedings_fake[:, 2*i+1], c='black', marker='+', s=16)
        plt.scatter(embedings[:, 2*i], embedings[:, 2*i+1], c=labels_expected, cmap='tab10', s=2)
        #plt.scatter(centroids_r[:,2*i], centroids_r[:,2*i+1], marker='X', c='green', s= 64)
        #plt.scatter(centroids_f[:,2*i], centroids_f[:,2*i+1], marker='o', c='red', s= 16)
        plt.scatter(centroids_core[:,2*i], centroids_core[:,2*i+1], marker='*', c='blue', s= 128)
        
        #plt.xlim(-2, 2)
        #plt.ylim(-2, 2)

    A = netD.core[0].get_transformation_matrix().detach().cpu().numpy()

    theta = np.linspace(0, 2*np.pi, 20)
    a, b = 1 * np.cos(theta), 1 * np.sin(theta)

    for i in range(A.shape[0]):
        h = A[i]
        t = [np.matmul(h, [x[0], x[1], 1]) for x in zip(a,b)]
        t = np.array(t)
        plt.plot(t[:, 0], t[:, 1])

    #plt.colorbar()
    plt.show()
    writer.add_figure('Embeddings', fig, epoch)

In [ ]:
for epoch in range(niter):

    netG.train()
    netD.train()
    report_loss_G = 0
    report_loss_D = 0
    report_loss_EV = 0
    local_count = 0
    for i, data in enumerate(tqdm(train_loader, desc='Training', disable=True)):
        
        # train with real
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        
        # netD.zero_grad()
        # core, dists_c, _, _ = netD(real_cpu)
        # dists_c.mean().backward()
        # optimizerD.step()
        
        netD.zero_grad()
        _, center_mass_dist, rand_dist = netD(real_cpu)
        rand_dist.mean().backward(retain_graph=True)
        center_mass_dist.mean().backward(retain_graph=True)
        ev_loss_r = keep_eigenvals_positive_loss(netD)
        ev_loss_r.backward()
        optimizerD.step()

        netD.zero_grad()
        core, _, _ = netD(real_cpu)
        errD_real = torch.square(core - 1).mean()
        errD_real.backward(retain_graph=True)
        ev_loss_r = keep_eigenvals_positive_loss(netD)
        ev_loss_r.backward()
        optimizerD.step()
        
        #train with fake
        netD.zero_grad()
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        core, _, _ = netD(fake.detach())
        errD_fake = torch.square(core).mean()
        errD_fake.backward(retain_graph=True)
        ev_loss_f = keep_eigenvals_positive_loss(netD)
        ev_loss_f.backward()
        optimizerD.step()
        
        #adjust sattelite
        #netD.zero_grad()
        # _, _, p, dp = netD(real_cpu)
        # dp.mean().backward()
        # optimizerD.step()
        
        #inp = torch.cat((fake.detach(), real_cpu))
        #target = torch.cat((torch.zeros(batch_size), torch.ones(batch_size))).to(device)
        
        # netD.zero_grad()
        # _, _, p, _ = netD(inp)
        # (torch.square(target - p).mean()).backward()
        # optimizerD.step()
        
        errD = errD_real + errD_fake
        
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        netD.zero_grad()
        
        core,_,_ = netD(fake)
        errG = torch.square(core - 1).mean()
        errG.backward()
        
        optimizerG.step()

        report_loss_G += errG.item()

        local_count += 1
        report_loss_D += errD.item()
        report_loss_EV += torch.max(ev_loss_f, ev_loss_r).item()
        
    netG.eval()
    netD.eval()

    losses = {}
    
    losses['EV'] = report_loss_EV / local_count
    losses['D'] = report_loss_D / local_count
    losses['G'] = report_loss_G / local_count
    writer.add_scalars('Loss', losses, epoch)
    fake = netG(fixed_noise)
    writer.add_images('Generated images', fake.detach(), epoch)
    draw_embeddings(epoch)    
    #mnist_distr, auc = get_test_arate_distr(netD)
    #mnist_distr_q = {}
    #mnist_distr_q["q20"] = np.quantile(mnist_distr, 0.2)
    #mnist_distr_q["q80"] = np.quantile(mnist_distr, 0.8)
    #writer.add_scalars("MNIST test  firings", mnist_distr_q, epoch)
    #writer.add_scalar("AUC", auc, epoch)
    print(f"Epoch {epoch}/{niter}")
    print(losses)

draw_embeddings(epoch)
num_gpu = 1 if torch.cuda.is_available() else 0

In [ ]:
netD.core[0].get_transformation_matrix()

In [ ]:
netD.core[0].get_transformation_matrix_eigenvals()

In [ ]:
# optimizerD = torch.optim.Adam(netD.parameters(), lr=0.002, betas=(0.5, 0.999))

In [ ]:
# netG.eval()
# netD.freeze_encoder()


# for epoch in range(niter):
   
#     netD.train()
#     report_loss_D = 0
#     report_loss_Rad = 0
#     report_loss_EV = 0.0
#     local_count = 0
#     for i, data in enumerate(tqdm(train_loader, desc='Training', disable=True)):
        
#         # train with real
#         real_cpu = data[0].to(device)
#         batch_size = real_cpu.size(0)
        
#         #netD.zero_grad()
#         #_, dist = netD.forward2(real_cpu)
#         #dist.abs().mean().backward()
#         #optimizerD.step()
        
#         noise = torch.randn(batch_size, nz, 1, 1, device=device)
#         fake = netG(noise)
        
#         inp = torch.cat((fake.detach(), real_cpu))
#         target = torch.cat((torch.zeros(batch_size), torch.ones(batch_size))).to(device)
        
#         netD.zero_grad()
#         firing, dist = netD(inp)
#         # _, _, p, _ = netD(inp)
#         #d = torch.where(dist>1, dist, 0.1)
#         errD = (torch.square(target - firing).mean()) #+ d.mean()
#         errD.backward(retain_graph=True)
#         ev_loss = keep_eigenvals_positive_loss(netD)
#         ev_loss.backward()
#         optimizerD.step()
        
#         # netD.zero_grad()
#         # firing, dist = netD.forward2(real_cpu)
#         # errD_real = torch.square(firing - 1).mean()
#         # errD_real.backward()
#         # optimizerD.step()
        
#         # #train with fake
#         # netD.zero_grad()
        
#         # firing, _ = netD.forward2(fake.detach())
#         # errD_fake = torch.square(firing).mean()
#         # errD_fake.backward()
#         # optimizerD.step()
        
#         #errD = errD_real + errD_fake
        
#         local_count += 1
#         #report_loss_Rad += dist.mean().item()
#         report_loss_D += errD.item()
#         if ev_loss.item() > report_loss_EV:
#             report_loss_EV = ev_loss.item()
        
#     netG.eval()
#     netD.eval()

#     losses = {}
    
#     losses['Adjusting D'] = report_loss_D / local_count
#     losses['Adjusting Rad'] = report_loss_Rad / local_count
#     losses['EV'] = report_loss_EV
    
    
#     writer.add_scalars('Loss', losses, epoch)
#     draw_embeddings(epoch)    
#     #mnist_distr, auc = get_test_arate_distr(netD)
#     #mnist_distr_q = {}
#     #mnist_distr_q["q20"] = np.quantile(mnist_distr, 0.2)
#     #mnist_distr_q["q80"] = np.quantile(mnist_distr, 0.8)
#     #writer.add_scalars("MNIST test  firings", mnist_distr_q, epoch)
#     #writer.add_scalar("AUC", auc, epoch)
#     print(f"Epoch {epoch}/{niter}")
#     print(losses)

## Валидация

In [ ]:
D = netD
G = netG
D.eval()
G.eval()

with torch.no_grad():
    batch_size = 49
    latent_size = 100
    
    fixed_noise = torch.randn(batch_size, latent_size, 1, 1)
    if torch.cuda.is_available():
        fixed_noise = fixed_noise.cuda()
    fake_images = G(fixed_noise)

    fake_images_np = fake_images.cpu().detach().numpy()
    fake_images_np = fake_images_np.reshape(fake_images_np.shape[0], 28, 28)
    R, C = 7, 7
    for i in range(batch_size):
        plt.subplot(R, C, i + 1)
        plt.imshow(fake_images_np[i], cmap='gray')
    plt.show()

In [ ]:
A = netD.core[0].get_transformation_matrix().detach().cpu().numpy()

theta = np.linspace(0, 2*np.pi, 20)
a, b = 1 * np.cos(theta), 1 * np.sin(theta)

for i in range(A.shape[0]):
    h = A[i]
    t = [np.matmul(h, [x[0], x[1], 1]) for x in zip(a,b)]
    t = np.array(t)
    plt.plot(t[:, 0], t[:, 1])
    
plt.show()
#c_real = lambda x: D.real(torch.FloatTensor([x]).to(device)).max(1).values.detach().cpu().item()
#c_core = lambda x: D.core(torch.FloatTensor([x]).to(device)).max(1).values.detach().cpu().item()
#f = lambda x: [[c_core(a), 0, 0] for a in x]

#xy = np.mgrid[-2:2:200j, -2:2:200j].reshape(2,-1).T

#img = np.reshape(f(xy),(200,200,3))
#plt.figure(figsize=(6, 6))
#plt.imshow(img, extent=[-2,2,-2,2])
#plt.show()

In [ ]:
A[0]

In [ ]:
def get_arate(inp):
    return D(inp)[1]
get_arate(inp.to(device))

In [ ]:
centroids_r = D.core[0].centroids.squeeze(-1).cpu().detach().numpy()
#centroids_f = D.fake[0].centroids.squeeze(-1).cpu().detach().numpy()
#centroids_r

In [ ]:
D.core[0].get_transformation_matrix()

In [ ]:
D.core[0].get_transformation_matrix_eigenvals()

In [ ]:
embedings_fake = []

with torch.no_grad():
    batch_size = 256
    latent_size = 100
    
    fixed_noise = torch.randn(batch_size, latent_size, 1, 1)
    if torch.cuda.is_available():
        fixed_noise = fixed_noise.cuda()
    fake_images = G(fixed_noise)
    embeding = D.main(fake_images)
    embedings_fake.append(embeding.cpu().numpy())

embedings_fake = np.concatenate(embedings_fake, axis=0)    

In [ ]:
embedings = []
labels_expected = []
with torch.no_grad():
    for data, target in tqdm(test_loader, desc='Encoding'):
        data = data.view((-1,1,28,28)).to(device)
        embeding = D.main(data)
        embedings.append(embeding.cpu().numpy())
        labels_expected.append(np.argmax(target.squeeze(1).cpu().numpy(), axis=1))
embedings = np.concatenate(embedings, axis=0)
labels_expected = np.concatenate(labels_expected, axis=0)
plt.figure(figsize=(6, 6))

R, C = 1, 1
cnt = 1
for i in range(1):
    plt.subplot(R, C, cnt)
    cnt += 1
    plt.scatter(embedings_fake[:, 2*i], embedings_fake[:, 2*i+1], c='black', marker='+', s=30)
    plt.scatter(embedings[:, 2*i], embedings[:, 2*i+1], c=labels_expected, cmap='tab10', s=2)
    plt.scatter(centroids_r[:,2*i], centroids_r[:,2*i+1], marker='x', c='green', s= 100)
    #plt.scatter(centroids_f[:,2*i], centroids_f[:,2*i+1], marker='o', c='red', s= 20)
    #plt.xlim(-0.5, 0.5)
    #plt.ylim(-0.5, 0.5)
#plt.colorbar()
plt.show()



In [ ]:
embedings = []
labels_expected = []
with torch.no_grad():
    for data, target in tqdm(emnist_loader, desc='Encoding'):
        data = data.view((-1,1,28,28)).to(device)
        embeding = D.main(data)
        embedings.append(embeding.cpu().numpy())
        labels_expected.append(target.cpu().numpy())
embedings = np.concatenate(embedings, axis=0)
labels_expected = np.concatenate(labels_expected, axis=0)
plt.figure(figsize=(6, 6))

R, C = 1, 1
cnt = 1
for i in range(1):
    plt.subplot(R, C, cnt)
    cnt += 1
    plt.scatter(embedings_fake[:, 2*i], embedings_fake[:, 2*i+1], c='black', marker='+', s=30)
    plt.scatter(embedings[:, 2*i], embedings[:, 2*i+1], c=labels_expected, cmap='tab10', s=2)
    plt.scatter(centroids_r[:,2*i], centroids_r[:,2*i+1], marker='x', c='green', s= 20)
    #plt.scatter(centroids_f[:,2*i], centroids_f[:,2*i+1], marker='o', c='red', s= 20)
    #plt.xlim(-0.5, 0.5)
    #plt.ylim(-0.5, 0.5)
#plt.colorbar()
plt.show()

In [ ]:
firing_levels = []
with torch.no_grad():
    for data, target in tqdm(test_loader, desc='Encoding'):
        data = data.view((-1,1,28,28)).to(device)
        rates = get_arate(data)
        #output = D.main(data)
        #r = D.real(output)
        #rates = r

        firing_levels.append(rates.cpu().numpy())
        
firing_levels = np.concatenate(firing_levels, axis=0)
fig = plt.figure(figsize =(10, 5))
plt.boxplot(firing_levels, notch=True, showfliers=True)
plt.show()

In [ ]:
firings_emnist = {}
for m in emnist_mapping:
    firings_emnist[emnist_mapping[m]] = []

with torch.no_grad():
    for data, labels in tqdm(emnist_loader, desc='Encoding'):
        data = data.view((-1, 1, 28, 28)).to(device) 
        arate = get_arate(data)
        #output = D.main(data)
        #r = D.real(output)
        #arate = 1 - r.sum(dim = 1)
        
        for label, flabel in zip(labels, arate.cpu().numpy()):
            firings_emnist[emnist_mapping[label.item()]].append(flabel)

In [ ]:
labels, data = firings_emnist.keys(), firings_emnist.values()
fig = plt.figure(figsize =(33, 5))
plt.boxplot(data, notch=True, showfliers=False)
plt.xticks(range(1, len(labels) + 1), labels)
plt.show()

In [ ]:
firings_mnist = {}
firings_mnist['MNIST'] = firing_levels
firings = {**firings_mnist, **firings_emnist} 
labels, data = firings.keys(), firings.values()

fig = plt.figure(figsize =(12, 2))
#plt.ylim(ymin=0)
plt.boxplot(data, notch=True, showfliers=False)
plt.xticks(range(1, len(labels) + 1), labels)
plt.show()

writer.add_figure('Anomaly Detection', fig)

In [ ]:
lab_true = []
lab_pred = []
for k,v in firings_mnist.items():
    for p in v:
        lab_true.append(1)
        lab_pred.append(p)
for k,v in firings_emnist.items():
    if k == 'Oo':
        continue
    for p in v:
        lab_true.append(0)
        lab_pred.append(p)

fpr, tpr, threshold = metrics.roc_curve(lab_true, lab_pred)
roc_auc = metrics.auc(fpr, tpr)

fig = plt.figure(figsize =(4, 4))
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
writer.add_figure('ROC', fig)